Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse

In [2]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points * 2654435761 + num_sets + density)
    sets = sparse.lil_array((num_sets, num_points), dtype = bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets - 1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [3]:
x = make_set_covering_problem(100, 100, .3)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: True


# Our Solution

# Set up

In [3]:
import numpy as np

from itertools import product
from random import random, randint, shuffle, seed, sample
from scipy import sparse
from copy import copy
from functools import reduce

In [4]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points * 2654435761 + num_sets + density)
    sets = sparse.lil_array((num_sets, num_points), dtype = bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets - 1), p] = True
    return sets

In [5]:
n = 100
PROBLEM_SIZE = NUM_SETS = n
x = make_set_covering_problem(n, n, .3)

SETS = x.toarray()
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: True


## Baseline

In [8]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

In [11]:
current_state = [False for _ in range(PROBLEM_SIZE)]
print(current_state)
print(fitness2(current_state))
count_fitness = 0
for step in range(10_000):
    new_state = tweak(current_state)
    if fitness2(new_state) > fitness2(current_state):
        current_state = new_state
        count_fitness += 1
        print(fitness2(current_state))
print(count_fitness)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
(0, 0)
(27, -1)
(50, -2)
(61, -3)
(73, -4)
(84, -5)
(89, -6)
(92, -7)
(94, -8)
(95, -9)
(98, -10)
(99, -11)
(100, -12)
(100, -11)
(100, -10)
(100, -9)
15


### Random N-1 Tweaking

In [12]:
def tweak(state):
    new_state = copy(state)
    n_swapping = randint(0, PROBLEM_SIZE - 1)
    index_list = sample(range(0, PROBLEM_SIZE - 1), n_swapping)
    new_state = [not new_state[i] if i in index_list else new_state[i] for i in range(len(new_state))]
    return new_state

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

In [13]:
current_state = [False for _ in range(PROBLEM_SIZE)]
print(current_state)
print(fitness2(current_state))
count_fitness = 0
for step in range(10_000):
    new_state = tweak(current_state)
    if fitness2(new_state) > fitness2(current_state):
        current_state = new_state
        print(fitness2(current_state))
        count_fitness += 1

print(count_fitness)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
(0, 0)
(78, -4)
(100, -72)
(100, -46)
(100, -43)
(100, -34)
(100, -33)
(100, -29)
(100, -26)
(100, -24)
(100, -23)
(100, -22)
(100, -21)
(100, -19)
(100, -18)
(100, -17)
(100, -16)
(100, -15)
(100, -14)
(100, -13)
(100, -12)
(100, -11)
(100, -10)
(100, -9)
23


### Random 2 Tweak

In [14]:
def tweak(state):
    new_state = copy(state)
    index_list = sample(range(0, PROBLEM_SIZE - 1), 2)
    new_state = [not new_state[i] if i in index_list else new_state[i] for i in range(len(new_state))]
    return new_state

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

In [18]:
current_state = [False for _ in range(PROBLEM_SIZE)]

print(fitness2(current_state))
count_fitness = 0
for step in range(10_000):
    new_state = tweak(current_state)
    
    if fitness2(new_state) > fitness2(current_state):
        current_state = new_state
        print(fitness2(current_state))
        count_fitness += 1

print(count_fitness)

(0, 0)
(49, -2)
(77, -4)
(88, -6)
(94, -8)
(97, -10)
(98, -12)
(99, -14)
(100, -16)
(100, -14)
(100, -12)
(100, -10)
11


### Random 3 tweak

In [19]:
def tweak(state):
    new_state = copy(state)
    index_list = sample(range(0, PROBLEM_SIZE - 1), 3)
    new_state = [not new_state[i] if i in index_list else new_state[i] for i in range(len(new_state))]
    return new_state

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

In [21]:
current_state = [False for _ in range(PROBLEM_SIZE)]

print(fitness2(current_state))
count_fitness = 0
for step in range(10_000):
    new_state = tweak(current_state)
    
    if fitness2(new_state) > fitness2(current_state):
        current_state = new_state
        print(fitness2(current_state))
        count_fitness += 1

print(count_fitness)

(0, 0)
(74, -3)
(91, -6)
(95, -9)
(96, -12)
(99, -15)
(100, -16)
(100, -15)
(100, -14)
(100, -13)
(100, -12)
(100, -11)
(100, -10)
(100, -9)
(100, -8)
14


### Simulated annealing

In [37]:
def simulated_annealing(current_state, new_state, temperature):

    return np.exp(-(fitness2(current_state)[1] - fitness2(new_state)[1])/(temperature + 0.0001))

def tweak(state):
    new_state = copy(state)
    index_list = sample(range(0, PROBLEM_SIZE - 1), 2)
    new_state = [not new_state[i] if i in index_list else new_state[i] for i in range(len(new_state))]
    return new_state

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost


In [53]:
current_state = [False for _ in range(PROBLEM_SIZE)]

temperature = 0.01
print(fitness2(current_state))
count_fitness = 0
for step in range(10_000):

    new_state = tweak(current_state)
    temperature = temperature * 0.5 if step % 100 == 0 else temperature

    if fitness2(new_state) > fitness2(current_state):

        current_state = new_state
        print(fitness2(current_state))
        count_fitness += 1
    
    else:
        
        p = simulated_annealing(current_state, new_state, temperature)
        
        if random() < p:
        
            current_state = new_state
            print("SA:", fitness2(current_state))
            count_fitness += 1

print(count_fitness)

(0, 0)
(46, -2)
(73, -4)
(85, -6)
(92, -8)
(97, -10)
(99, -12)
1.0
SA: (99, -12)
(100, -14)
4.880209537149898e-171
4.880209537149898e-171
4.880209537149898e-171
1.0
SA: (100, -14)
4.880209537149898e-171
4.880209537149898e-171
4.880209537149898e-171
(100, -12)
1.0
SA: (100, -12)
4.880209537149898e-171
4.880209537149898e-171
4.880209537149898e-171
4.880209537149898e-171
1.0
SA: (99, -12)
(100, -14)
1.0
SA: (100, -14)
4.880209537149898e-171
4.880209537149898e-171
4.880209537149898e-171
4.880209537149898e-171
4.880209537149898e-171
1.0
SA: (100, -14)
4.880209537149898e-171
4.880209537149898e-171
1.0
SA: (100, -14)
1.0
SA: (99, -14)
4.880209537149898e-171
1.0
SA: (99, -14)
4.880209537149898e-171
1.0
SA: (99, -14)
1.0
SA: (99, -14)
(100, -16)
1.0
SA: (100, -16)
4.880209537149898e-171
4.880209537149898e-171
1.0
SA: (100, -16)
4.880209537149898e-171
4.880209537149898e-171
4.880209537149898e-171
4.880209537149898e-171
4.880209537149898e-171
4.880209537149898e-171
4.880209537149898e-171
4.880209

C:\Users\Mattia\AppData\Local\Temp\ipykernel_14004\2093145647.py:3: RuntimeWarning: overflow encountered in exp
  return np.exp(-(fitness2(current_state)[1] - fitness2(new_state)[1])/(temperature + 0.0001))


inf
SA: (97, -12)
(100, -14)
0.0
0.0
0.0
0.0
0.0
1.0
SA: (100, -14)
1.0
SA: (100, -14)
0.0
0.0
0.0
0.0
1.0
SA: (100, -14)
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
SA: (99, -14)
0.0
0.0
0.0
(100, -16)
0.0
0.0
0.0
0.0
1.0
SA: (100, -16)
1.0
SA: (100, -16)
1.0
SA: (100, -16)
1.0
SA: (100, -16)
0.0
0.0
1.0
SA: (100, -16)
0.0
0.0
0.0
1.0
SA: (100, -16)
1.0
SA: (100, -16)
0.0
1.0
SA: (100, -16)
0.0
1.0
SA: (100, -16)
0.0
0.0
0.0
0.0
0.0
0.0
1.0
SA: (100, -16)
0.0
0.0
0.0
1.0
SA: (100, -16)
1.0
SA: (100, -16)
0.0
1.0
SA: (100, -16)
0.0
1.0
SA: (100, -16)
0.0
0.0
0.0
0.0
1.0
SA: (99, -16)
0.0
0.0
0.0
(100, -16)
1.0
SA: (100, -16)
0.0
0.0
(100, -14)
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
SA: (100, -14)
0.0
0.0
0.0
1.0
SA: (99, -14)
(100, -16)
1.0
SA: (100, -16)
1.0
SA: (99, -16)
0.0
0.0
1.0
SA: (99, -16)
(100, -18)
0.0
0.0
1.0
SA: (100, -18)
0.0
1.0
SA: (100, -18)
0.0
0.0
1.0
SA: (100, -18)
1.0
SA: (100, -18)
0.0
0.0
0.0
1.0
SA: (100, -18)
0.0
0.0
1.0
SA: (100, -18)
0.0
1.0
SA: (100, -18)
(100, -16)
1.0
SA: (9